In [53]:
import torch
from torchvision import transforms, datasets, models
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import helper

In [54]:
trainTransform = transforms.Compose([transforms.RandomRotation(30),
                                    transforms.RandomResizedCrop(100),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor()])

testTransform = transforms.Compose([transforms.RandomRotation(30),
                                   transforms.RandomResizedCrop(100),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor()])

In [55]:
root = 'Cat_Dog_Data'

trainSet = datasets.ImageFolder(root=root+'/train', transform=trainTransform)
testSet = datasets.ImageFolder(root=root+'/test', transform=testTransform)

In [56]:
trainLoader = torch.utils.data.DataLoader(dataset=trainSet, batch_size=64, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=testSet, batch_size=64, shuffle=False)

#### For CUDA
- 1. model.cuda() for GPU
- 2. model.cpu() for CPU
- 3. torch.cuda.is_available() yes/no

In [64]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Using DenseNet121

In [65]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

#### 1. Freeze the feature layers(n-1)

In [66]:
for param in model.parameters():
    param.requires_grad = False #Turn off the gradients
    

from collections import OrderedDict
import torch.nn as nn

classifer = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu',nn.ReLU()),
    ('fc2',nn.Linear(500,2)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = classifer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
model.to(device);

In [76]:
epochs =10
steps =0
running_loss = 0
print_every =5

for epoch in range(epochs):
    for images, labels in trainLoader:
        steps +=1
        
        #images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if step % print_every ==0:
            model.eval()
            test_loss = 0
            accuracy = 0
                
            for images, labels, in testLoader:
                images, labels = images.to(device), labels.to(device)
                logps = model(images)
                loss = criterion(logps, labels)
                test_loss += loss.item()
                    
                #calculate the accuracy
                ps = torch.exp(logps)
                top_ps, top_class = ps.topk(1, dim=1)
                equality = top_class == label.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
                    
                print(f"Epoch {epoch+1}/{epochs}.."
                      f"Train Loss: {running_loss/print_every: .3f}.."
                      f"Test loss: {test_loss/len(testLoader):.3f}.."
                      f"Test accuracy: {accuracy/len(testLoader):.3f}")
                
                
                #RESTORE THE MODEL TO TRAINMODE
                running_loss = 0
                model.train()



RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [68]:
model = model.den
    

AttributeError: 'DenseNet' object has no attribute 'den'

In [52]:
cuda = torch.cuda.is_available()
cuda

False